<a href="https://colab.research.google.com/github/npasuparthi/nagendra-AI_ML/blob/master/NLP_Sarcasm_Detect_Nagendra_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1>What is Colaboratory?</h1>

Colaboratory, or "Colab" for short, allows you to write and execute Python in your browser, with 
- Zero configuration required
- Free access to GPUs
- Easy sharing

Whether you're a **student**, a **data scientist** or an **AI researcher**, Colab can make your work easier. Watch [Introduction to Colab](https://www.youtube.com/watch?v=inN8seMm7UI) to learn more, or just get started below!

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import pandas as pd
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [33]:
data = pd.read_json('/content/drive//MyDrive/SarcasmDetect/Sarcasm_Headlines_Dataset.json', lines= True)

In [34]:
data.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [35]:
# Import stopwords with scikit-learn
from sklearn.feature_extraction import text
stop = text.ENGLISH_STOP_WORDS

In [36]:
#remove stop words
data['headline'] = data['headline'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))



In [37]:
samples = data['headline'].to_numpy()
labels = data['is_sarcastic'].to_numpy()

In [39]:
samples[43]

"new google project delivers critical info refugees' smartphones"

In [40]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

In [41]:
#Create a vocabulary index

from tensorflow.keras.layers import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [43]:
# top 50 words in the vocabulary are 
vectorizer.get_vocabulary()[:50]


['',
 '[UNK]',
 'new',
 'trump',
 'man',
 'just',
 'report',
 'woman',
 'area',
 'day',
 'says',
 'donald',
 'time',
 'us',
 'like',
 'trumps',
 'people',
 'house',
 'obama',
 'white',
 'life',
 'make',
 'women',
 'world',
 'clinton',
 'years',
 'americans',
 'way',
 'family',
 'study',
 'its',
 'gop',
 'black',
 '5',
 'nation',
 'police',
 'really',
 'know',
 'good',
 'cant',
 'death',
 'finds',
 'watch',
 'the',
 'school',
 'big',
 'going',
 'american',
 'home',
 'best']

In [48]:
#get length of vocabulary

print(vectorizer.vocabulary_size()) 
# putput is 200k, same as max_tokens set into the tokenizer

20000


In [59]:
# dictionary mapping words to their indices:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [50]:
#download glove embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2021-09-25 10:17:36--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-09-25 10:17:36--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-09-25 10:17:37--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [53]:
pwd

'/content'

In [54]:
ls

drive/             glove.6B.200d.txt  glove.6B.50d.txt  sample_data/
glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip


In [58]:
import os

#Create a dictionary mapping words (strings) to their NumPy vector representation:
path_to_glove_file = os.path.join(
    os.path.expanduser("~"), "/content/glove.6B.100d.txt"
)

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [60]:
#create embedding matrix

num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 18084 words (1916 misses)


In [61]:
#create Embedding layer
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [63]:
#Building the LSTM Model
model_lstm = tf.keras.Sequential([
    embedding_layer,
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [64]:
model_lstm.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         2000200   
_________________________________________________________________
bidirectional (Bidirectional (None, 32)                14976     
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 2,015,381
Trainable params: 15,181
Non-trainable params: 2,000,200
_________________________________________________________________


In [65]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

In [67]:
model_lstm.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)
model_lstm.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
179/179 [==============================] - 23s 81ms/step - loss: 0.6046 - accuracy: 0.6789 - val_loss: 0.5430 - val_accuracy: 0.7260
Epoch 2/20
179/179 [==============================] - 14s 76ms/step - loss: 0.5073 - accuracy: 0.7538 - val_loss: 0.5116 - val_accuracy: 0.7451
Epoch 3/20
179/179 [==============================] - 14s 76ms/step - loss: 0.4728 - accuracy: 0.7762 - val_loss: 0.4874 - val_accuracy: 0.7608
Epoch 4/20
179/179 [==============================] - 14s 76ms/step - loss: 0.4456 - accuracy: 0.7915 - val_loss: 0.4779 - val_accuracy: 0.7645
Epoch 5/20
179/179 [==============================] - 14s 76ms/step - loss: 0.4253 - accuracy: 0.8041 - val_loss: 0.4728 - val_accuracy: 0.7702
Epoch 6/20
179/179 [==============================] - 14s 76ms/step - loss: 0.4067 - accuracy: 0.8144 - val_loss: 0.4595 - val_accuracy: 0.7765
Epoch 7/20
179/179 [==============================] - 14s 76ms/step - loss: 0.3920 - accuracy: 0.8218 - val_loss: 0.4555 - val_accuracy: